<a href="https://colab.research.google.com/github/yeb2Binfang/sklearn_learning/blob/main/RandomForest/RandomForest_Fillna.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

我们现实生活中的数据，总是会有一些缺失值，这时，random forest是一个好方法来填补这些缺失值。

我们一般的做法有时就删掉，有时就补0，有时就补均值等。

在sklearn中，我们可以使用sklearn.impute.SimpleInputer来轻松地将均值，中值，或者其他最常用的数值填补到数据中。

在这个案例中，我们将使用均值，0，和随机森林回归来填补缺失值，并验证下拟合状况，来找出对使用的数据集的最佳填补缺失值的办法

##导包

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import load_boston
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score


##导入数据，看看数据有什么

In [5]:
boston_data = load_boston().data
print(boston_data.shape)

X, y = boston_data, load_boston().target
print(X[:5])
print(y[:5])

(506, 13)
[[6.3200e-03 1.8000e+01 2.3100e+00 0.0000e+00 5.3800e-01 6.5750e+00
  6.5200e+01 4.0900e+00 1.0000e+00 2.9600e+02 1.5300e+01 3.9690e+02
  4.9800e+00]
 [2.7310e-02 0.0000e+00 7.0700e+00 0.0000e+00 4.6900e-01 6.4210e+00
  7.8900e+01 4.9671e+00 2.0000e+00 2.4200e+02 1.7800e+01 3.9690e+02
  9.1400e+00]
 [2.7290e-02 0.0000e+00 7.0700e+00 0.0000e+00 4.6900e-01 7.1850e+00
  6.1100e+01 4.9671e+00 2.0000e+00 2.4200e+02 1.7800e+01 3.9283e+02
  4.0300e+00]
 [3.2370e-02 0.0000e+00 2.1800e+00 0.0000e+00 4.5800e-01 6.9980e+00
  4.5800e+01 6.0622e+00 3.0000e+00 2.2200e+02 1.8700e+01 3.9463e+02
  2.9400e+00]
 [6.9050e-02 0.0000e+00 2.1800e+00 0.0000e+00 4.5800e-01 7.1470e+00
  5.4200e+01 6.0622e+00 3.0000e+00 2.2200e+02 1.8700e+01 3.9690e+02
  5.3300e+00]]
[24.  21.6 34.7 33.4 36.2]


In [6]:
n_samples = X.shape[0]
n_features = X.shape[1]


##为完整数据集放入缺失值

In [9]:
#首先，我们得先去确定下我们希望放入的缺失值的比例，在这里，我们假设是50%，原先我们有506*13 = 6578个数据，我们假设是50%，那么，就会有50%的数据进行替换补缺

rng = np.random.RandomState(0) #就是random state
missing_rate = 0.5
n_missing_samples = int(np.floor(n_samples * n_features * missing_rate))
print(n_missing_samples)

3289


In [10]:
#所有数据要随机遍布在数据集的各行各列中，而一个缺失的数据需要行列索引，然后在这个索引中填值
#如果能够创造一个数组，包含3289个分布在0~506中间的行索引，和3289个分布在0~13之间的索引，那么我们就可以利用索引来为数据中的任意3289个位置赋空值
#然后，我们用0， 均值，随机森林来填写这些缺失值，然后查看回归的结果如何

#产生3289个[0~13)的数值
missing_features = rng.randint(0,n_features,n_missing_samples)
print(missing_features)
missing_samples - rng.randint(0,n_samples,n_missing_samples)

#我们现在采样了3289个数据，远远超过我们的样本量506，所以我们使用随机抽取的函数randint。但如果我们需要
#的数据量小于我们的样本量506，那我们可以采用np.random.choice来抽样，choice会随机抽取不重复的随机数，
#因此可以帮助我们让数据更加分散，确保数据不会集中在一些行中
#missing_samples = rng.choice(dataset.data.shape[0],n_missing_samples,replace=False)


[12  5  0 ... 11  0  2]
